In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# =====================================================
# LOAD DATA
# =====================================================
train_df = pd.read_csv("train2.csv")       # contains features incl. returns
labels_df = pd.read_csv("labels.csv")     # contains Backward_Bin, Forward_Bin

# Features (remove non-numeric or irrelevant columns as in your main notebook)
X = train_df.select_dtypes(include=[np.number]).copy()
X = X.drop(columns=["Percent_change_forward", "Percent_change_backward"], errors="ignore")

# Tomorrow state (0-based)
y_all = labels_df["Forward_Bin"].values - 1
n_states = int(y_all.max()) + 1

# =====================================================
# TRAIN/VAL/TEST SPLIT (time-based or random — here random for baseline)
# =====================================================
X_train, X_temp, y_train, y_temp = train_test_split(X, y_all, test_size=0.30, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=False)

# =====================================================
# STANDARDIZE FEATURES
# =====================================================
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std   = scaler.transform(X_val)
X_test_std  = scaler.transform(X_test)

n_features = X_train_std.shape[1]

# =====================================================
# DATASET & LOADERS
# =====================================================
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(SimpleDataset(X_train_std, y_train), batch_size=256, shuffle=True)
val_loader   = DataLoader(SimpleDataset(X_val_std,   y_val),   batch_size=512, shuffle=False)
test_loader  = DataLoader(SimpleDataset(X_test_std,  y_test),  batch_size=512, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =====================================================
# CLASS WEIGHTS
# =====================================================
class_counts = np.bincount(y_train, minlength=n_states)
class_weights = class_counts.sum() / np.maximum(class_counts, 1)
class_weights = class_weights / class_weights.mean()
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

# =====================================================
# BASELINE MODEL
# =====================================================
class BaselineNet(nn.Module):
    def __init__(self, n_features, n_states):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, 128),
            nn.ReLU(),
            nn.Linear(128, n_states)
        )
    def forward(self, x):
        return self.net(x)

baseline = BaselineNet(n_features, n_states).to(device)
optimizer_b = torch.optim.Adam(baseline.parameters(), lr=1e-3)
criterion_b = nn.CrossEntropyLoss(weight=class_weights)

# =====================================================
# TRAINING LOOP
# =====================================================
def run_epoch(model, loader, train=False):
    model.train() if train else model.eval()
    total_loss, total_correct, total_n = 0, 0, 0

    with torch.set_grad_enabled(train):
        for Xb, yb in loader:
            Xb = Xb.to(device)
            yb = yb.to(device)

            logits = model(Xb)
            loss = criterion_b(logits, yb)

            if train:
                optimizer_b.zero_grad()
                loss.backward()
                optimizer_b.step()

            preds = logits.argmax(dim=1)
            total_correct += (preds == yb).sum().item()
            total_loss += loss.item() * len(Xb)
            total_n += len(Xb)

    return total_loss / total_n, total_correct / total_n

best_val = float("inf")
best_state = None

for epoch in range(1, 16):
    tr_loss, tr_acc = run_epoch(baseline, train_loader, train=True)
    va_loss, va_acc = run_epoch(baseline, val_loader, train=False)

    if va_loss < best_val:
        best_val = va_loss
        best_state = {k: v.cpu().clone() for k, v in baseline.state_dict().items()}

    if epoch % 5 == 0 or epoch == 1:
        print(f"[Baseline] Epoch {epoch:02d} | train {tr_loss:.3f}/{tr_acc:.3f} | val {va_loss:.3f}/{va_acc:.3f}")

baseline.load_state_dict(best_state)
baseline.to(device)

# =====================================================
# TEST EVAL
# =====================================================
test_loss, test_acc = run_epoch(baseline, test_loader, train=False)

print("\n=== BASELINE TEST RESULTS ===")
print(f"Baseline test loss: {test_loss:.4f}")
print(f"Baseline test acc : {test_acc:.4f}")

# =====================================================
# CONFUSION MATRIX
# =====================================================
baseline.eval()
all_true, all_pred = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb = Xb.to(device)
        logits = baseline(Xb)
        preds = logits.argmax(dim=1).cpu().numpy()
        all_pred.append(preds)
        all_true.append(yb.numpy())

all_true = np.concatenate(all_true)
all_pred = np.concatenate(all_pred)

cm_baseline = confusion_matrix(all_true, all_pred, labels=np.arange(n_states))
print(cm_baseline.shape)  # should now be (55, 55)


[Baseline] Epoch 01 | train 4.009/0.022 | val 4.087/0.021
[Baseline] Epoch 05 | train 3.609/0.046 | val 4.275/0.030
[Baseline] Epoch 10 | train 3.441/0.052 | val 4.447/0.030
[Baseline] Epoch 15 | train 3.340/0.060 | val 4.551/0.024

=== BASELINE TEST RESULTS ===
Baseline test loss: 4.1939
Baseline test acc : 0.0382
(55, 55)
